In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.signal import stft
from PIL import Image
from tqdm import tqdm
import shutil
from collections import defaultdict
from scipy.ndimage import gaussian_filter
import random
import pandas as pd

# =============== Reproducibility ===============
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
set_seed(42)

# =============== Parametreler ===============
fs = 10240
start_idx = int(1.5 * fs)
end_idx   = int(4.5 * fs)
segment_length = 1024
hop = 512

# Split oranları: ~70% train / 10% validation / 20% test
train_ratio = 0.7
val_ratio   = 0.1  # kalan %0.2 test olacak

# Zaman aralığı (reassigned fonksiyonu içinde kullanılıyor)
Ts = 1 / fs

# =============== Reassignment method ===============
def reassigned_spectrogram(x, fs, window='hann', nperseg=512, noverlap=384, nfft=1024):
    # Standard STFT
    f, t, Zxx = stft(x, fs=fs, window=window, nperseg=nperseg,
                     noverlap=noverlap, nfft=nfft, return_onesided=True)

    # Meshgrid
    T, F = np.meshgrid(t, f)

    # Time derivative (finite difference)
    t_derivative = np.zeros_like(Zxx, dtype=np.complex128)
    t_derivative[:, 1:-1] = (Zxx[:, 2:] - Zxx[:, :-2]) / (t[2] - t[0])
    t_derivative[:, 0]    = (Zxx[:, 1] - Zxx[:, 0])   / (t[1] - t[0])
    t_derivative[:, -1]   = (Zxx[:, -1] - Zxx[:, -2]) / (t[-1] - t[-2])

    # Frequency derivative (finite difference)
    f_derivative = np.zeros_like(Zxx, dtype=np.complex128)
    f_derivative[1:-1, :] = (Zxx[2:, :] - Zxx[:-2, :]) / (f[2:, np.newaxis] - f[:-2, np.newaxis])
    f_derivative[0,  :]   = (Zxx[1,  :] - Zxx[0,  :])  / (f[1] - f[0])
    f_derivative[-1, :]   = (Zxx[-1, :] - Zxx[-2, :])  / (f[-1] - f[-2])

    power = np.abs(Zxx)**2

    # Reassignment operators
    mask = (np.abs(Zxx) > 1e-10)
    t_reassigned = np.copy(T)
    f_reassigned = np.copy(F)

    if np.any(mask):
        t_reassigned[mask] = T[mask] - np.real(np.conj(Zxx[mask]) * t_derivative[mask]
                                               / (2j * np.pi * fs * np.abs(Zxx[mask])**2))
        f_reassigned[mask] = F[mask] + np.real(np.conj(Zxx[mask]) * f_derivative[mask]
                                               / (2j * np.pi * Ts * np.abs(Zxx[mask])**2))

    f_reassigned = np.clip(f_reassigned, f[0], f[-1])

    # Daha pürüzsüz görselleştirme için bin sayısını artır
    t_bins = np.linspace(np.min(t), np.max(t), len(t) * 2)
    f_bins = np.linspace(np.min(f), np.max(f), len(f) * 2)
    dt = t_bins[1] - t_bins[0]
    df = f_bins[1] - f_bins[0]

    reassigned_spec = np.zeros((len(f_bins)-1, len(t_bins)-1), dtype=np.float32)

    # Enerjiyi reassigned koordinatlara biriktir
    for i in range(len(f)):
        for j in range(len(t)):
            if mask[i, j]:
                t_idx = int((t_reassigned[i, j] - t_bins[0]) / dt)
                f_idx = int((f_reassigned[i, j] - f_bins[0]) / df)
                if 0 <= t_idx < len(t_bins)-1 and 0 <= f_idx < len(f_bins)-1:
                    reassigned_spec[f_idx, t_idx] += power[i, j].astype(np.float32)

    reassigned_spec = gaussian_filter(reassigned_spec, sigma=1.0)
    return f_bins[:-1], t_bins[:-1], reassigned_spec

# =============== Yollar ===============
root_dir   = r"C:\Users\VICTUS\OneDrive\Masaüstü\EECSL"
output_dir = r"C:\Users\VICTUS\Pictures\(1.5-4.5s)\Reassigned_New(1.5-4.5s)"

# Temiz başlangıç
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# Sınıflar
labels = ['healthy', 'imbalance', 'loose', 'misalignment']

# =============== Segment toplama ===============
class_segments = defaultdict(list)

for label in labels:
    folder_path = os.path.join(root_dir, f"{label}_start")
    mat_files = [f for f in os.listdir(folder_path) if f.endswith(".mat")]
    for file in tqdm(mat_files, desc=f"Yükleniyor: {label}"):
        path = os.path.join(folder_path, file)
        data = loadmat(path)

        # cur1/cur2 seçimi
        if 'cur1' in file.lower():
            signal_key = 'cur1'
        elif 'cur2' in file.lower():
            signal_key = 'cur2'
        else:
            # dosya adında cur1/cur2 ayrımı yoksa anahtar kontrolü yap
            if 'cur1' in data:
                signal_key = 'cur1'
            elif 'cur2' in data:
                signal_key = 'cur2'
            else:
                continue  # uygun anahtar yok

        if signal_key not in data:
            continue

        signal = data[signal_key].squeeze().astype(np.float64)
        # Z-normalize + kesit (1.5–4.5s)
        signal = (signal - np.mean(signal)) / (np.std(signal) + 1e-8)
        signal = signal[start_idx:end_idx]

        # Kaydırmalı segmentler
        for j in range(0, len(signal) - segment_length + 1, hop):
            segment = signal[j:j + segment_length]
            # (basename, key, j, segment)
            class_segments[label].append((file[:-4], signal_key, j, segment))

# Her sınıftan eşit sayıda segment
min_count = min(len(v) for v in class_segments.values())
print(f"Eşit segment sayısı (her sınıf için): {min_count}")

# =============== Split ve görsel üretimi ===============
# Çıktı klasörlerini hazırla
for sp in ['train', 'validation', 'test']:
    for lb in labels:
        os.makedirs(os.path.join(output_dir, sp, lb), exist_ok=True)

index_rows = []  # CSV index map için

for label, segments in class_segments.items():
    # deterministik karışım
    rng = np.random.RandomState(42)
    rng.shuffle(segments)
    selected = segments[:min_count]

    n_train = int(train_ratio * min_count)
    n_val   = int(val_ratio * min_count)
    n_test  = min_count - n_train - n_val

    split_map = {
        'train'     : selected[:n_train],
        'validation': selected[n_train:n_train+n_val],
        'test'      : selected[n_train+n_val:]
    }

    for split, segment_list in split_map.items():
        save_dir = os.path.join(output_dir, split, label)
        os.makedirs(save_dir, exist_ok=True)

        for (basename, key, j, segment) in tqdm(segment_list, desc=f"İşleniyor: {split}/{label}"):
            # Reassigned spectrogram
            f_axis, t_axis, Sxx = reassigned_spectrogram(segment, fs=fs, window='hann',
                                                         nperseg=512, noverlap=384, nfft=1024)

            Sxx_db = 10 * np.log10(Sxx + 1e-10)
            vmin = np.percentile(Sxx_db, 5)
            vmax = np.percentile(Sxx_db, 95)

            # 224x224 çıktı için doğrudan kaydet (figsize=2.24, dpi=100)
            fig, ax = plt.subplots(figsize=(2.24, 2.24), dpi=100)
            ax.axis('off')
            ax.pcolormesh(t_axis, f_axis, Sxx_db, shading='gouraud',
                          cmap='jet', vmin=vmin, vmax=vmax)
            ax.set_ylim(0, 2000)
            plt.tight_layout(pad=0)

            # benzersiz & deterministik dosya adı (1D eşleşmesi için önemli)
            # örn: healthy__MOTOR1_cur1__512.png
            safe_base = basename.replace(" ", "_")
            filename = f"{label}__{safe_base}__{key}__{j}.png"
            save_path = os.path.join(save_dir, filename)
            fig.savefig(save_path, bbox_inches='tight', pad_inches=0)
            plt.close(fig)

            # indeks haritasına ekle
            index_rows.append({
                'split'     : split,
                'label'     : label,
                'filename'  : filename,
                'basename'  : safe_base,
                'signal_key': key,
                'seg_start' : j
            })

# =============== İndeks CSV ===============
index_df = pd.DataFrame(index_rows)
csv_out = os.path.join(root_dir, "reassigned_index_map.csv")
index_df.to_csv(csv_out, index=False, encoding='utf-8-sig')
print(f"\nReassigned spektrogram dataset oluşturma tamamlandı!")
print(f"İndeks haritası kaydedildi: {csv_out}")
print(f"Çıktı klasörü: {output_dir}")


Yükleniyor: misalignment: 100%|██████████| 8/8 [00:00<00:00, 47.15it/s]


Eşit segment sayısı (her sınıf için): 472


İşleniyor: test/misalignment: 100%|██████████| 95/95 [00:21<00:00,  4.45it/s]


Reassigned spektrogram dataset oluşturma tamamlandı!
İndeks haritası kaydedildi: C:\Users\VICTUS\OneDrive\Masaüstü\EECSL\reassigned_index_map.csv
Çıktı klasörü: C:\Users\VICTUS\Pictures\(1.5-4.5s)\Reassigned_New(1.5-4.5s)
